In [121]:
from bs4 import BeautifulSoup as bs 
from urllib.request import urlopen
from tqdm import tqdm
import pandas as pd 
import numpy as np
import requests
import pyttsx3

In [12]:
url = 'https://www.airlinequality.com/airline-reviews/air-arabia#google_vignette'

# Set up headers to mimic a real browser
headers = {
    'User -Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.google.com/'
}

# Make the request with the headers
response = requests.get(url, headers=headers)

#### **connecting to the website** 

In [13]:
soup = bs(response.content, 'html.parser')

In [15]:
container = soup.find_all('article',{'itemprop':'review'})
len(container)

10

In [ ]:
# resume
a = container[0].find_all('h2',{'class':'text_header'})
print(a[0].text)

"still waiting for delay compensation"


In [92]:
text_content = container[0].find_all('div',{"class": "text_content"})
text_content[0].text

'✅ Trip Verified |   Terrible service, still waiting for delay compensation 7 months later. My flight was delayed by 5 hours. No staff at Gatwick airport to provide information or refreshments. Terrible service over the phone upon returning. Take weeks/months for a response and still waiting.'

In [77]:
trs = container[0].find_all('table',{'class':'review-ratings'})
len(a)

1

In [80]:
trs

[<table class="review-ratings">
 <tr><td class="review-rating-header type_of_traveller">Type Of Traveller</td><td class="review-value">Solo Leisure</td></tr>
 <tr><td class="review-rating-header cabin_flown">Seat Type</td><td class="review-value">Economy Class</td></tr>
 <tr><td class="review-rating-header route">Route</td><td class="review-value">Gatwick to Tangier</td></tr>
 <tr><td class="review-rating-header date_flown">Date Flown</td><td class="review-value">August 2024</td></tr>
 <tr>
 <td class="review-rating-header seat_comfort">Seat Comfort</td>
 <td class="review-rating-stars stars"><span class="star fill">1</span><span class="star">2</span><span class="star">3</span><span class="star">4</span><span class="star">5</span></td>
 </tr>
 <tr>
 <td class="review-rating-header cabin_staff_service">Cabin Staff Service</td>
 <td class="review-rating-stars stars"><span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star">4</span>

In [83]:
headers = trs[0].find_all('td')

In [86]:
headers

[<td class="review-rating-header type_of_traveller">Type Of Traveller</td>,
 <td class="review-value">Solo Leisure</td>,
 <td class="review-rating-header cabin_flown">Seat Type</td>,
 <td class="review-value">Economy Class</td>,
 <td class="review-rating-header route">Route</td>,
 <td class="review-value">Gatwick to Tangier</td>,
 <td class="review-rating-header date_flown">Date Flown</td>,
 <td class="review-value">August 2024</td>,
 <td class="review-rating-header seat_comfort">Seat Comfort</td>,
 <td class="review-rating-stars stars"><span class="star fill">1</span><span class="star">2</span><span class="star">3</span><span class="star">4</span><span class="star">5</span></td>,
 <td class="review-rating-header cabin_staff_service">Cabin Staff Service</td>,
 <td class="review-rating-stars stars"><span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star">4</span><span class="star">5</span></td>,
 <td class="review-rating-header f

In [ ]:
# Extract all table rows
review_data = []  
current_review = {}

text_conten = container[0].find_all('div',{"class": "text_content"})
current_review["text_content"]=text_conten[0].text.strip()

resume = container[0].find_all('h2',{'class':'text_header'})
current_review["resume"]=resume[0].text.strip()

if "Trip Verified" in str(current_review):
    current_review["verified"] = True
else:
    current_review["verified"] = False



rows = soup.find_all("tr")
 
# Loop through rows and gather data
for row in rows:
    header = row.find("td", class_="review-rating-header")
    value = row.find("td", class_="review-value")
    stars = row.find("td", class_="review-rating-stars")

    if header:
        key = header.text.strip()

        # Store value if found
        if value:
            current_review[key] = value.text.strip()

        # Count stars if found
        elif stars:
            filled_stars = len(stars.find_all("span", class_="star fill"))
            current_review[key] = f"{filled_stars} stars"
        
        # Default to "N/A" if no value or stars
        else:
            current_review[key] = "N/A"

# Once done, store the review data
review_data.append(current_review)

# Output the list of review dictionaries
data = pd.DataFrame(review_data)

{'text_content': '✅ Trip Verified |   Terrible service, still waiting for delay compensation 7 months later. My flight was delayed by 5 hours. No staff at Gatwick airport to provide information or refreshments. Terrible service over the phone upon returning. Take weeks/months for a response and still waiting.', 'resume': '"still waiting for delay compensation"', 'verified': True, 'Food & Beverages': '2 stars', 'Inflight Entertainment': '1 stars', 'Seat Comfort': '2 stars', 'Staff Service': '2 stars', 'Value for Money': '2 stars', 'Type Of Traveller': 'Solo Leisure', 'Seat Type': 'Economy Class', 'Route': 'Shiraz to Kuala lumpur via Sharjah', 'Date Flown': 'April 2024', 'Cabin Staff Service': '1 stars', 'Ground Service': '1 stars', 'Value For Money': '1 stars', 'Recommended': 'no', 'Wifi & Connectivity': '1 stars', 'Aircraft': 'A320'}


In [99]:
data = pd.DataFrame(review_data)
data

,text_content,resume,verified,Food & Beverages,Inflight Entertainment,Seat Comfort,Staff Service,Value for Money,Type Of Traveller,Seat Type,Route,Date Flown,Cabin Staff Service,Ground Service,Value For Money,Recommended,Wifi & Connectivity,Aircraft
0,"✅ Trip Verified | Terrible service, still wa...","""still waiting for delay compensation""",True,2 stars,1 stars,2 stars,2 stars,2 stars,Solo Leisure,Economy Class,Shiraz to Kuala lumpur via Sharjah,April 2024,1 stars,1 stars,1 stars,no,1 stars,A320


In [146]:
engine = pyttsx3.init()
engine.setProperty('rate', 150)  
engine.setProperty('volume', 0.3)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

# **scrapping programme**

In [148]:

def connect(p):
    url = f"https://www.airlinequality.com/airline-reviews/air-arabia/page/{p}/"
    headers = {
        'User -Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.google.com/'
    }

    # Make the request with the headers
    response = requests.get(url, headers=headers)
    soup = bs(response.content, "html.parser")
   
    return soup




In [150]:
pages = 100
review_data = []
for i in tqdm(range(pages),desc=f"processing pages {i+1}"):
    connect(i+1)
    engine.say(f'scraping page number {i+1}')
    engine.runAndWait()
    containers = soup.find_all('article',{'itemprop':'review'})

    for i in tqdm(range(len(containers))):  
        current_review = {}

        text_conten = containers[i].find_all('div',{"class": "text_content"})
        current_review["text_content"]=text_conten[0].text.strip()

        resume = containers[i].find_all('h2',{'class':'text_header'})
        current_review["resume"]=resume[0].text.strip()

        if "Trip Verified" in str(current_review):
            current_review["verified"] = True
        else:
            current_review["verified"] = False



        rows = containers[i].find_all("tr")
        
        # Loop through rows and gather data
        for row in rows:
            header = row.find("td", class_="review-rating-header")
            value = row.find("td", class_="review-value")
            stars = row.find("td", class_="review-rating-stars")

            if header:
                key = header.text.strip()

                # Store value if found
                if value:
                    current_review[key] = value.text.strip()

                # Count stars if found
                elif stars:
                    filled_stars = len(stars.find_all("span", class_="star fill"))
                    current_review[key] = f"{filled_stars} stars"
                
                # Default to "N/A" if no value or stars
                else:
                    current_review[key] = "N/A"

        # Once done, store the review data
        review_data.append(current_review)

data = pd.DataFrame(review_data)
        

processing pages 15: 100%|██████████| 100/100 [06:23<00:00,  3.83s/it]


In [152]:
data.shape

(1000, 16)

In [154]:
data

,text_content,resume,verified,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Value For Money,Recommended,Inflight Entertainment,Wifi & Connectivity,Aircraft
0,"✅ Trip Verified | Terrible service, still wa...","""still waiting for delay compensation""",True,Solo Leisure,Economy Class,Gatwick to Tangier,August 2024,1 stars,3 stars,2 stars,1 stars,1 stars,no,NaN,NaN,NaN
1,Not Verified | I recently had a very disappoin...,“highly restrictive and unfair policy”,False,Solo Leisure,Economy Class,Kathmandu to Abu Dhabi,January 2025,2 stars,2 stars,3 stars,1 stars,1 stars,no,1 stars,2 stars,NaN
2,✅ Trip Verified | Terrible experience with thi...,"""Better off avoiding this airline""",True,Couple Leisure,Economy Class,Athens to Bangkok,January 2025,1 stars,1 stars,1 stars,1 stars,1 stars,no,1 stars,1 stars,A321
3,✅ Trip Verified | The Worst Airline you coul...,"""deduct a hefty 50% of the ticket""",True,Solo Leisure,Economy Class,Sharjah to Kuala Lumpur,January 2025,NaN,NaN,NaN,NaN,1 stars,no,NaN,NaN,NaN
4,Not Verified | Before two years I used to fly ...,“far behind that quality I enjoy”,False,Solo Leisure,Economy Class,Kochi to Abu Dhabi,December 2024,1 stars,3 stars,3 stars,5 stars,3 stars,yes,1 stars,1 stars,A320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,✅ Trip Verified | Really bad experience. I d...,"2Really bad experience""",True,Solo Leisure,Economy Class,Sharjah to Amman,November 2024,1 stars,2 stars,1 stars,1 stars,1 stars,no,NaN,1 stars,A320
996,Not Verified | I had a terrible experience wi...,"""I had a terrible experience""",False,Family Leisure,Economy Class,Dubai to Tbilisi,June 2024,1 stars,1 stars,1 stars,1 stars,1 stars,no,1 stars,1 stars,A320
997,Not Verified | You can't imagine how bad their...,"""about 5-6 hours delayed""",False,Family Leisure,Economy Class,Chittagong to Milan via Sharjah,July 2024,1 stars,1 stars,1 stars,1 stars,2 stars,no,1 stars,1 stars,NaN
998,Not Verified | Very bad customer service at Tr...,“Poor service”,False,Family Leisure,Economy Class,Thiruvananthapuram to Abha,June 2024,2 stars,1 stars,2 stars,1 stars,1 stars,no,1 stars,1 stars,NaN


In [156]:
data['Aircraft'].isna().sum()

np.int64(600)